In [262]:
from pathlib import Path
from dotenv import dotenv_values, load_dotenv
import os

env_path = Path("/Users/samuelminer/Projects/nissan_options/wheel_strategy/.env")

print("Parsed keys:", dotenv_values(env_path).keys())

load_dotenv(env_path, override=True)
print("os.getenv:", bool(os.getenv("DATABENTO_API_KEY")))


Parsed keys: odict_keys(['DATABENTO_API_KEY'])
os.getenv: True


In [263]:
import sys
sys.executable

from dotenv import load_dotenv
import os

load_dotenv()  # loads .env from current working directory

assert os.getenv("DATABENTO_API_KEY"), "DATABENTO_API_KEY still not found"



In [264]:
import os
import pandas as pd
import databento as db
import pandas_market_calendars as mcal

client = db.Historical()


### Import Daily Equity Data

In [265]:
import databento as db
import pandas as pd

# Uses DATABENTO_API_KEY from environment
client = db.Historical()

symbol = "TSLA"
dataset = "EQUS.MINI"     # consolidated US equities (best choice)
schema = "ohlcv-1d"       # DAILY bars
tz = "America/New_York"
days = 252*2

end = pd.Timestamp.utcnow().normalize() - pd.Timedelta(days=1)
start = end - pd.Timedelta(days=252*2)

data = client.timeseries.get_range(
    dataset=dataset,
    symbols=symbol,
    schema=schema,
    stype_in="raw_symbol",
    start=start,
    end=end,
)


/var/folders/6k/0v57cgbd2k37vp0lh44zby640000gn/T/ipykernel_22992/507646123.py:16: BentoWarning: The streaming request contained one or more days which have reduced quality: 2025-03-24 (degraded), 2025-04-04 (degraded), 2025-05-06 (degraded)... See: https://databento.com/docs/api-reference-historical/metadata/metadata-get-dataset-condition
  data = client.timeseries.get_range(


In [266]:
df = data.to_df(tz=tz)#.reset_index(names="date").set_index('date',inplace=True)


### Create Equity Technical Filter

In [267]:
import pandas as pd

df_bb = df.copy().sort_index()

window = 20
k = 2.0  # 2-sigma Bollinger Bands

# Rolling stats on close
roll = df_bb["close"].rolling(window=window, min_periods=window)
df_bb["sma20"] = roll.mean()
df_bb["std20"] = roll.std(ddof=0)

# Bollinger Bands
df_bb["bb_upper"] = df_bb["sma20"] + k * df_bb["std20"]
df_bb["bb_lower"] = df_bb["sma20"] - k * df_bb["std20"]

# Optional: Bollinger %B and Bandwidth
df_bb["bb_pctb"] = (df_bb["close"] - df_bb["bb_lower"]) / (df_bb["bb_upper"] - df_bb["bb_lower"])
df_bb["bb_bandwidth"] = (df_bb["bb_upper"] - df_bb["bb_lower"]) / df_bb["sma20"]

# # Keep the most useful columns for strategy work
# df_bb_out = df_bb[["date", "symbol", "open", "high", "low", "close", "volume",
#                    "sma20", "bb_upper", "bb_lower", "bb_pctb", "bb_bandwidth"]]

# df_bb_out.tail(30)


In [268]:
df_bb.dropna().head()

,rtype,publisher_id,instrument_id,open,high,low,close,volume,symbol,sma20,std20,bb_upper,bb_lower,bb_pctb,bb_bandwidth
ts_event,,,,,,,,,,,,,,,
2024-08-22 20:00:00-04:00,35,95,16244,213.64,221.48,212.60,220.72,2287138,TSLA,211.9840,12.078469,236.140937,187.827063,0.680818,0.227913
2024-08-25 20:00:00-04:00,35,95,16244,220.21,220.54,211.01,213.30,1370757,TSLA,211.0330,11.153204,233.339407,188.726593,0.550815,0.211402
2024-08-26 20:00:00-04:00,35,95,16244,213.29,215.63,206.97,208.06,1135520,TSLA,210.1870,10.695524,231.578048,188.795952,0.450283,0.203543
2024-08-27 20:00:00-04:00,35,95,16244,209.50,211.80,202.22,202.37,1516810,TSLA,208.6800,9.500665,227.681330,189.678670,0.333959,0.182110
2024-08-28 20:00:00-04:00,35,95,16244,206.64,214.88,205.70,207.24,1434799,TSLA,208.1845,9.302352,226.789204,189.579796,0.474617,0.178733


### Equity Technical Filter

In [269]:
# Equity Filters
df_equity_entry = df_bb.copy()[['close','sma20','bb_lower']].dropna()
df_equity_entry['sma_entry'] = df_equity_entry['close'] <= df_equity_entry['sma20']
df_equity_entry['bb_entry'] = df_equity_entry['close'] <= df_equity_entry['bb_lower']
df_equity_entry[['sma_entry']].value_counts()
df_equity_entry[['bb_entry']].value_counts()
df_equity_entry.head()

,close,sma20,bb_lower,sma_entry,bb_entry
ts_event,,,,,
2024-08-22 20:00:00-04:00,220.72,211.9840,187.827063,False,False
2024-08-25 20:00:00-04:00,213.30,211.0330,188.726593,False,False
2024-08-26 20:00:00-04:00,208.06,210.1870,188.795952,True,False
2024-08-27 20:00:00-04:00,202.37,208.6800,189.678670,True,False
2024-08-28 20:00:00-04:00,207.24,208.1845,189.579796,True,False


In [270]:
df_equity_entry[df_equity_entry['sma_entry'] | df_equity_entry['bb_entry']].index.tolist()[:5]

[Timestamp('2024-08-26 20:00:00-0400', tz='America/New_York'),
 Timestamp('2024-08-27 20:00:00-0400', tz='America/New_York'),
 Timestamp('2024-08-28 20:00:00-0400', tz='America/New_York'),
 Timestamp('2024-09-05 20:00:00-0400', tz='America/New_York'),
 Timestamp('2024-10-06 20:00:00-0400', tz='America/New_York')]

### Get Options Data For Dates that Pass Technical Filter

In [407]:
import databento as db
import pandas as pd

client = db.Historical()

dataset = "OPRA.PILLAR"
schema = "cmbp-1"
tz = "America/New_York"

start = pd.Timestamp("2023-06-06 15:45", tz=tz)
end   = start + pd.Timedelta(minutes=1)

data = client.timeseries.get_range(
    dataset=dataset,
    schema=schema,
    symbols=f"{symbol}.OPT",     # ✅ parent symbology format
    stype_in="parent",           # ✅ parent lookup
    start=start,
    end=end,
)

df_opts = data.to_df(tz=tz).sort_values("ts_event")
df_opts.head()


,ts_event,rtype,publisher_id,instrument_id,action,side,price,size,flags,ts_in_delta,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_pb_00,ask_pb_00,symbol
ts_recv,,,,,,,,,,,,,,,,,
2023-06-06 15:45:00.000069556-04:00,2023-06-06 15:44:59.999864576-04:00,177,30,738198022,A,A,5.05,84,194,0,5.00,5.05,150,84,0,0,TSLA 230616P00215000
2023-06-06 15:45:00.000074115-04:00,2023-06-06 15:44:59.999868928-04:00,177,30,687866628,A,B,19.05,176,194,0,19.05,19.25,176,142,0,0,TSLA 240315C00300000
2023-06-06 15:45:00.000095165-04:00,2023-06-06 15:44:59.999889664-04:00,177,30,738198042,A,A,5.75,15,194,0,5.65,5.75,355,15,0,0,TSLA 230609P00222500
2023-06-06 15:45:00.000095264-04:00,2023-06-06 15:44:59.999890176-04:00,177,30,704643949,A,B,28.20,21,194,0,28.20,28.40,21,118,0,0,TSLA 230616C00192500
2023-06-06 15:45:00.000098559-04:00,2023-06-06 15:44:59.999894016-04:00,177,30,687866390,A,B,11.25,117,194,0,11.25,11.30,117,24,0,0,TSLA 230721C00235000


In [ ]:
# start = pd.Timestamp("2023-06-06 15:45", tz=tz)
# end   = start + pd.Timedelta(minutes=1)

# data = client.timeseries.get_range(
#     dataset="OPRA.PILLAR",
#     schema="ohlcv-1m",
#     symbols="AAPL.OPT",
#     stype_in="parent",
#     start=start,          # 15:45
#     end=end,              # 15:46
# )

# df_ohlc = data.to_df(tz="America/New_York")


In [ ]:
# def parse_opra_symbol(sym: str):
#     # Example: "AAPL 240119P00205000"
#     root, rest = sym.split()
#     expiration = pd.to_datetime(rest[:6], format="%y%m%d")
#     call_put = rest[6]
#     strike = int(rest[7:]) / 1000
#     return expiration, call_put, strike

# df_opts[["expiration", "call_put", "strike"]] = (
#     df_opts["symbol"]
#     .apply(parse_opra_symbol)
#     .apply(pd.Series)
# )

In [ ]:
# # ============================================================================
# # Fetch Option Definitions for a Single Day
# # ============================================================================

# def fetch_option_symbols(ticker, date):
#     """Fetch all available option symbols for a ticker on a specific date"""
#     print(f"Fetching option symbols for {ticker} on {date}...")
    
#     date_dt = pd.Timestamp(date)
#     start_time = pd.Timestamp(date_dt.date()).tz_localize('America/New_York').replace(hour=15, minute=45)
#     end_time = start_time + pd.Timedelta(minutes=1)
    
#     print(f"  Query time: {start_time}")
    
#     try:
#         data = client.timeseries.get_range(
#             dataset='OPRA.PILLAR',
#             schema='ohlcv-1m',
#             symbols=f'{ticker}.OPT',
#             stype_in='parent',
#             start=start_time,
#             end=end_time,
#         )
        
#         df = data.to_df(tz='America/New_York')
#         print(f"  ✓ Fetched {len(df)} option contracts")
        
#         return df
        
#     except Exception as e:
#         print(f"  ✗ Error: {e}")
#         import traceback
#         traceback.print_exc()
#         return pd.DataFrame()

# def parse_expiration_from_symbol(symbol):
#     """Extract just the expiration date from OSI symbol"""
#     try:
#         parts = symbol.split()
#         if len(parts) != 2:
#             return None
        
#         rest = parts[1]
#         date_str = rest[:6]  # YYMMDD
        
#         year = 2000 + int(date_str[:2])
#         month = int(date_str[2:4])
#         day = int(date_str[4:6])
        
#         return pd.Timestamp(year, month, day)
#     except:
#         return None

# def parse_call_put_from_symbol(symbol):
#     """Extract call/put indicator from OSI symbol"""
#     try:
#         parts = symbol.split()
#         if len(parts) != 2:
#             return None
        
#         rest = parts[1]
#         return rest[6]  # 'C' or 'P'
#     except:
#         return None

# def calculate_dte_trading_days(entry_date, exp_date, nyse_calendar):
#     """Calculate trading days between entry and expiration"""
#     entry_dt = pd.Timestamp(entry_date).normalize()
#     exp_dt = pd.Timestamp(exp_date).normalize()
    
#     schedule = nyse_calendar.schedule(start_date=entry_dt, end_date=exp_dt)
#     sessions = schedule.index
#     dte = len(sessions[(sessions > entry_dt) & (sessions <= exp_dt)])
    
#     return dte

# # ============================================================================
# # EXECUTE
# # ============================================================================

# ticker = 'TSLA'
# date = '2023-06-06'

# # Step 1: Fetch option symbols
# df_options = fetch_option_symbols(ticker, date)

# if len(df_options) == 0:
#     print("⚠️ No data returned")
# else:
#     # Step 2: Parse symbols
#     df_options['expiration'] = df_options['symbol'].apply(parse_expiration_from_symbol)
#     df_options['call_put'] = df_options['symbol'].apply(parse_call_put_from_symbol)
    
#     # Step 3: Calculate DTE (trading days)
#     entry_date = df_options.index[0].tz_localize(None).normalize()
    
#     # Get unique expirations and calculate DTE for each
#     unique_expirations = df_options['expiration'].unique()
#     dte_map = {}
    
#     print(f"\nCalculating DTE (trading days) from {entry_date.date()}...")
#     for exp_date in unique_expirations:
#         dte = calculate_dte_trading_days(entry_date, exp_date, nyse)
#         dte_map[exp_date] = dte
    
#     # Map DTE to all rows
#     df_options['dte'] = df_options['expiration'].map(dte_map)
    
#     print(f"DTE distribution:")
#     print(df_options.groupby('dte')['symbol'].count().sort_index())
    
#     # Step 4: Filter by DTE and call/put
#     df_filtered = df_options[
#         (df_options['dte'] >= 30) &
#         (df_options['dte'] <= 45) &
#         (df_options['call_put'] == 'P')
#     ]
    
#     print(f"\nFiltered from {len(df_options)} to {len(df_filtered)} options")
#     print(f"  (Puts only, 30-45 TRADING days to expiry)")
    
#     print(f"\nDTE in filtered set:")
#     print(df_filtered['dte'].value_counts().sort_index())
    
#     # Step 5: Fetch cmbp-1 data for filtered symbols
#     symbols_to_fetch = df_filtered['symbol'].tolist()
    
#     print(f"\nFetching market data for {len(symbols_to_fetch)} filtered symbols...")
    
#     start = pd.Timestamp("2023-06-06 15:45", tz='America/New_York')
#     end = start + pd.Timedelta(minutes=1)
    
#     data = client.timeseries.get_range(
#         dataset='OPRA.PILLAR',
#         schema='cmbp-1',
#         symbols=symbols_to_fetch,
#         stype_in='raw_symbol',
#         start=start,
#         end=end,
#     )
    
#     df_opts = data.to_df(tz='America/New_York').sort_values("ts_event")
    
#     print(f"✓ Fetched {len(df_opts)} option contracts")
#     print(f"\nSample:")
#     print(df_opts[['symbol', 'bid_px_00', 'ask_px_00', 'bid_sz_00', 'ask_sz_00']].head(10))


Fetching option symbols for TSLA on 2023-06-06...
  Query time: 2023-06-06 15:45:00-04:00
  ✓ Fetched 315 option contracts

Calculating DTE (trading days) from 2023-06-06...
DTE distribution:
dte
3      160
8       57
12      22
17       8
21       1
26       2
31      35
51       5
70       4
95       3
156      6
262      6
325      1
406      3
638      2
Name: symbol, dtype: int64

Filtered from 315 to 14 options
  (Puts only, 30-45 TRADING days to expiry)

DTE in filtered set:
dte
31    14
Name: count, dtype: int64

Fetching market data for 14 filtered symbols...
✓ Fetched 2485 option contracts

Sample:
                                                    symbol  bid_px_00  \
ts_recv                                                                 
2023-06-06 15:45:00.000115529-04:00  TSLA  230721P00220000      16.05   
2023-06-06 15:45:00.004306566-04:00  TSLA  230721P00220000      16.05   
2023-06-06 15:45:00.004434923-04:00  TSLA  230721P00215000      13.60   
2023-06-06 15:45:00

In [408]:
sym = df_opts["symbol"]

# Split ROOT and OPRA code (e.g. "AAPL" and "240119P00205000")
root_and_code = sym.str.split(expand=True)
df_opts["root"] = root_and_code[0]
code = root_and_code[1]

# Expiration: YYMMDD in positions 0–5
df_opts["expiration"] = pd.to_datetime(code.str[:6], format="%y%m%d")

# Call/Put flag: single char at position 6
df_opts["call_put"] = code.str[6]

# Strike: remaining digits, usually in 1/1000 dollars
# Example: "00205000" -> 205.000
strike_int = code.str[7:].astype("int32")
df_opts["strike"] = strike_int / 1000.0


In [390]:
# df = df_opts.copy()

# # Get NYSE calendar (or use 'NASDAQ')
# nyse = mcal.get_calendar('NYSE')

# # For each row, calculate valid trading days
# def get_trading_days(row):
#     event_date = row.name.tz_convert("America/New_York").date()
#     exp_date = pd.to_datetime(row["expiration"]).date()
    
#     # Get valid trading days between dates
#     schedule = nyse.valid_days(start_date=event_date, end_date=exp_date)
#     return len(schedule) - 1  # -1 to exclude start date

# df["dte"] = df.apply(get_trading_days, axis=1)


In [409]:
import numpy as np

df = df_opts.copy()

nyse = mcal.get_calendar("NYSE")

# 1) Extract event dates (from index) in NY time, then drop tz to match schedule
event_dt = df.index.tz_convert("America/New_York").normalize()
event_days = pd.to_datetime(event_dt.date)   # tz-naive midnight dates

# 2) Extract expiration dates as plain dates (no tz)
exp_dt = pd.to_datetime(df["expiration"])
exp_days = pd.to_datetime(exp_dt.dt.date)    # tz-naive midnight dates

# 3) Build ONE trading calendar over the full range
start_date = event_days.min().date()
end_date   = exp_days.max().date()

schedule = nyse.valid_days(start_date=start_date, end_date=end_date)
schedule = pd.to_datetime(schedule).normalize().tz_localize(None)  # tz-naive

# 4) Integer index for each trading day
cal_index = pd.Series(
    data=np.arange(len(schedule), dtype=int),
    index=schedule
)

# 5) Map event & expiry dates to trading-day indices
event_idx = cal_index.reindex(event_days).to_numpy()
exp_idx   = cal_index.reindex(exp_days).to_numpy()

# 6) DTE = (# trading days between) = expiry_index - event_index - 1
df["dte"] = (exp_idx - event_idx - 1).astype(int)


In [410]:
filtered_df = df[(df['dte'] >= 30) & (df['dte'] <= 45) & (df['call_put'] == 'P')].sort_values(['dte', 'strike'])
filtered_df.head()

,ts_event,rtype,publisher_id,instrument_id,action,side,price,size,flags,ts_in_delta,...,bid_sz_00,ask_sz_00,bid_pb_00,ask_pb_00,symbol,root,expiration,call_put,strike,dte
ts_recv,,,,,,,,,,,,,,,,,,,,,
2023-06-06 15:45:05.934932006-04:00,2023-06-06 15:45:05.934723328-04:00,177,30,721420930,A,A,0.01,319,194,0,...,0,319,0,0,TSLA 230721P00005000,TSLA,2023-07-21,P,5.0,30
2023-06-06 15:45:06.137894941-04:00,2023-06-06 15:45:06.137686784-04:00,177,30,721420930,A,A,0.01,442,194,0,...,0,442,0,0,TSLA 230721P00005000,TSLA,2023-07-21,P,5.0,30
2023-06-06 15:45:11.065068895-04:00,2023-06-06 15:45:11.064861952-04:00,177,30,721420930,A,A,0.01,341,194,0,...,0,341,0,0,TSLA 230721P00005000,TSLA,2023-07-21,P,5.0,30
2023-06-06 15:45:11.103310944-04:00,2023-06-06 15:45:11.103102976-04:00,177,30,721420930,A,A,0.01,258,194,0,...,0,258,0,0,TSLA 230721P00005000,TSLA,2023-07-21,P,5.0,30
2023-06-06 15:45:11.348062323-04:00,2023-06-06 15:45:11.347855104-04:00,177,30,721420930,A,A,0.01,319,194,0,...,0,319,0,0,TSLA 230721P00005000,TSLA,2023-07-21,P,5.0,30


In [411]:
filtered_df.symbol.unique()

array(['TSLA  230721P00005000', 'TSLA  230721P00010000',
       'TSLA  230721P00015000', 'TSLA  230721P00020000',
       'TSLA  230721P00025000', 'TSLA  230721P00030000',
       'TSLA  230721P00035000', 'TSLA  230721P00040000',
       'TSLA  230721P00045000', 'TSLA  230721P00050000',
       'TSLA  230721P00055000', 'TSLA  230721P00060000',
       'TSLA  230721P00065000', 'TSLA  230721P00070000',
       'TSLA  230721P00075000', 'TSLA  230721P00080000',
       'TSLA  230721P00085000', 'TSLA  230721P00090000',
       'TSLA  230721P00095000', 'TSLA  230721P00100000',
       'TSLA  230721P00105000', 'TSLA  230721P00110000',
       'TSLA  230721P00115000', 'TSLA  230721P00120000',
       'TSLA  230721P00125000', 'TSLA  230721P00130000',
       'TSLA  230721P00135000', 'TSLA  230721P00140000',
       'TSLA  230721P00145000', 'TSLA  230721P00150000',
       'TSLA  230721P00155000', 'TSLA  230721P00160000',
       'TSLA  230721P00165000', 'TSLA  230721P00170000',
       'TSLA  230721P00175000',

In [412]:
unique_timestamps = filtered_df.index.unique()
unique_symbols = filtered_df['symbol'].str.split().str[0].unique()  # Extract underlying symbols (e.g., 'AAPL' from 'AAPL 230707P00180000')

print(f"Unique timestamps: {len(unique_timestamps)}")
print(f"Unique underlying symbols: {unique_symbols}")
print(f"Date range: {unique_timestamps.min()} to {unique_timestamps.max()}")


Unique timestamps: 11319
Unique underlying symbols: ['TSLA']
Date range: 2023-06-06 15:45:00.000115529-04:00 to 2023-06-06 15:45:59.996597640-04:00


In [413]:
# Get unique timestamps from your filtered options
unique_timestamps = filtered_df.index.unique()

# For a specific moment, you can use the exact timestamp
# Since your data is at 2023-06-06 15:45:00-04:00, we'll fetch a narrow window around it
start_time = pd.Timestamp("2023-06-06 15:45:00", tz="US/Eastern")
end_time = pd.Timestamp("2023-06-06 15:46:00", tz="US/Eastern")  # 1 minute window

print(f"Fetching equity data for: {start_time} to {end_time}")

# Fetch OHLCV data for AAPL at the specific timestamp
equity_data = client.timeseries.get_range(
    dataset='XNAS.ITCH',  # NASDAQ for AAPL
    symbols=[f'{symbol}'],
    schema='ohlcv-1m',  # 1-minute OHLCV bars
    start=start_time,
    end=end_time,
    stype_in='raw_symbol'
)

# Convert to dataframe
equity_df = equity_data.to_df()
print(f"Fetched {len(equity_df)} equity records")
equity_df
# Or if you want to fetch for all unique timestamps in your filtered data:
# Get the earliest and latest timestamps from your options data
# start_time = unique_timestamps.min()
# end_time = unique_timestamps.max() + pd.Timedelta(minutes=1)  # Add 1 minute since end is exclusive

# equity_data = client.timeseries.get_range(
#     dataset='XNAS.ITCH',
#     symbols=['AAPL'],
#     schema='ohlcv-1m',
#     start=start_time,
#     end=end_time,
#     stype_in='raw_symbol'
# )


Fetching equity data for: 2023-06-06 15:45:00-04:00 to 2023-06-06 15:46:00-04:00
Fetched 1 equity records


,rtype,publisher_id,instrument_id,open,high,low,close,volume,symbol
ts_event,,,,,,,,,
2023-06-06 19:45:00+00:00,33,2,10274,219.75,219.91,219.75,219.86,19083,TSLA


In [414]:
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal

def add_trading_dte(df: pd.DataFrame, tz="America/New_York", cal_name="NYSE") -> pd.DataFrame:
    """
    Adds trading-days-to-expiration (DTE) using a single calendar build + vectorized mapping.
    Assumes df has columns: ts_event (tz-aware) and expiration (date/datetime).
    """
    out = df.copy()

    cal = mcal.get_calendar(cal_name)

    # Use event timestamp (not ts_recv index)
    event_dt = pd.to_datetime(out["ts_event"]).dt.tz_convert(tz).dt.normalize()
    event_days = pd.to_datetime(event_dt.dt.date)  # tz-naive midnight dates

    exp_dt = pd.to_datetime(out["expiration"])
    exp_days = pd.to_datetime(exp_dt.dt.date)      # tz-naive midnight dates

    start_date = event_days.min().date()
    end_date   = exp_days.max().date()

    schedule = cal.valid_days(start_date=start_date, end_date=end_date)
    schedule = pd.to_datetime(schedule).normalize().tz_localize(None)

    cal_index = pd.Series(np.arange(len(schedule), dtype=np.int32), index=schedule)

    event_idx = cal_index.reindex(event_days).to_numpy()
    exp_idx   = cal_index.reindex(exp_days).to_numpy()

    out["dte"] = (exp_idx - event_idx - 1).astype(np.int16)  # safe nullable int
    return out

df_opts = add_trading_dte(df_opts)

In [415]:
import numpy as np
import pandas as pd
from py_vollib.black_scholes.implied_volatility import implied_volatility
from py_vollib.black_scholes.greeks.analytical import delta

r = 0.04  # fixed risk-free rate

# 0) Keep only rows that actually have a quote (bid/ask)
quotes = df_opts[df_opts["bid_px_00"].notna() & df_opts["ask_px_00"].notna()].copy()

# 1) Compute mid price per tick
quotes["mid"] = (quotes["bid_px_00"] + quotes["ask_px_00"]) / 2

# 2) Collapse to ONE row per option contract (snapshot at ~3:45 pm)
chain_snapshot = (
    quotes
    .sort_values("ts_event")   # important: so tail(1) is the latest
    .groupby(["symbol", "expiration", "strike", "call_put"])
    .tail(1)                   # last quote for each contract
    .copy()
)
underlying_price = equity_df["close"].iloc[0]   # 15:45 close
chain_snapshot["underlying_last"] = underlying_price


In [416]:
def compute_iv(row):
    price = row["mid"]
    S     = row["underlying_last"]
    K     = row["strike"]
    t     = row["dte"] / 365.0
    flag  = "p" if row["call_put"] == "P" else "c"

    if not (np.isfinite(price) and np.isfinite(S) and np.isfinite(K) and t > 0):
        return np.nan
    if price <= 0 or S <= 0 or K <= 0:
        return np.nan

    try:
        return implied_volatility(price, S, K, t, r, flag)
    except Exception:
        return np.nan


def compute_delta(row):
    sigma = row["iv"]
    if not np.isfinite(sigma):
        return np.nan

    S    = row["underlying_last"]
    K    = row["strike"]
    t    = row["dte"] / 365.0
    flag = "p" if row["call_put"] == "P" else "c"

    return delta(flag, S, K, t, r, sigma)

chain_snapshot["iv"] = chain_snapshot.apply(compute_iv, axis=1)
chain_snapshot["delta"] = chain_snapshot.apply(compute_delta, axis=1)

chain_snapshot.head()

,ts_event,rtype,publisher_id,instrument_id,action,side,price,size,flags,ts_in_delta,...,symbol,root,expiration,call_put,strike,dte,mid,underlying_last,iv,delta
ts_recv,,,,,,,,,,,,,,,,,,,,,
2023-06-06 15:45:01.605961671-04:00,2023-06-06 15:45:01.605758976-04:00,177,30,738198142,A,B,91.00,283,194,0,...,TSLA 240621P00296670,TSLA,2024-06-21,P,296.67,261,91.850,219.86,0.597986,-0.611467
2023-06-06 15:45:05.776101520-04:00,2023-06-06 15:45:05.775898368-04:00,177,30,738198072,A,B,78.35,296,194,0,...,TSLA 240621P00280000,TSLA,2024-06-21,P,280.00,261,79.050,219.86,0.593048,-0.569216
2023-06-06 15:45:06.086457167-04:00,2023-06-06 15:45:06.086249984-04:00,177,30,721420438,A,B,0.01,235,194,0,...,TSLA 240119P00016670,TSLA,2024-01-19,P,16.67,155,0.015,219.86,1.305485,-0.000253
2023-06-06 15:45:06.258672230-04:00,2023-06-06 15:45:06.258470144-04:00,177,30,738197865,A,B,0.75,100,194,0,...,TSLA 231020P00095000,TSLA,2023-10-20,P,95.00,94,0.760,219.86,0.894101,-0.017912
2023-06-06 15:45:06.783183888-04:00,2023-06-06 15:45:06.782980864-04:00,177,30,687866519,A,A,13.65,184,194,0,...,TSLA 250117C00510000,TSLA,2025-01-17,C,510.00,405,13.325,219.86,0.654640,0.208671


In [417]:
def calculate_21dte_dates_vectorized(expirations, nyse_calendar):
    """
    Vectorized calculation of dates 21 trading days before expiration
    
    Args:
        expirations: Series of expiration dates
        nyse_calendar: NYSE calendar object
    
    Returns:
        Series of dates 21 trading days before expiration
    """
    # Get min/max range
    min_exp = expirations.min()
    max_exp = expirations.max()
    
    # Get all trading sessions in the range (with extra lookback)
    start_date = min_exp - pd.Timedelta(days=60)  # 60 calendar days = ~40 trading days
    end_date = max_exp
    
    schedule = nyse_calendar.schedule(start_date=start_date, end_date=end_date)
    trading_days = schedule.index
    
    # For each expiration, find the date 21 trading days before
    results = []
    for exp in expirations:
        exp_dt = pd.Timestamp(exp).normalize()
        
        # Get all trading days up to and including expiration
        valid_days = trading_days[trading_days <= exp_dt]
        
        # Go back 21 trading days
        if len(valid_days) >= 21:
            target_date = valid_days[-21]
        else:
            target_date = valid_days[0] if len(valid_days) > 0 else exp_dt
        
        results.append(target_date)
    
    return pd.Series(results, index=expirations.index)

# Use it
chain_snapshot['date_21dte'] = calculate_21dte_dates_vectorized(
    chain_snapshot['expiration'], 
    nyse)



In [418]:
chain_snapshot['date'] = chain_snapshot['ts_event'].dt.date

candidates = chain_snapshot[
    (chain_snapshot["call_put"] == "P")
    & chain_snapshot["dte"].between(30, 45)
    & chain_snapshot["delta"].abs().between(0.25, 0.35)
].copy()

candidates[["symbol", "expiration", "strike", "dte", "iv", "delta",'mid']].sort_values(
    ["dte", "strike"]
)


,symbol,expiration,strike,dte,iv,delta,mid
ts_recv,,,,,,,
2023-06-06 15:45:59.584316824-04:00,TSLA 230721P00200000,2023-07-21,200.0,30,0.671809,-0.272615,7.850
2023-06-06 15:45:59.592987302-04:00,TSLA 230721P00205000,2023-07-21,205.0,30,0.665605,-0.315837,9.525


In [419]:
candidates['hold_days'] = candidates['dte'] - 21
# candidates[''] = candidates[candidates['hold_days'] > 0]


In [420]:
backtest_candidates = candidates.copy()
backtest_candidates['cost_basis'] = backtest_candidates['underlying_last']*100 - backtest_candidates['mid']
backtest_candidates['premium'] = backtest_candidates['mid']
backtest_candidates['exit_50_perc'] = 0.5 * backtest_candidates['premium']
backtest_candidates = backtest_candidates[['symbol','date_21dte', 'cost_basis', 'premium', 'exit_50_perc','date','dte','expiration','mid','strike']]
backtest_candidates

,symbol,date_21dte,cost_basis,premium,exit_50_perc,date,dte,expiration,mid,strike
ts_recv,,,,,,,,,,
2023-06-06 15:45:59.584316824-04:00,TSLA 230721P00200000,2023-06-22,21978.150,7.850,3.9250,2023-06-06,30,2023-07-21,7.850,200.0
2023-06-06 15:45:59.592987302-04:00,TSLA 230721P00205000,2023-06-22,21976.475,9.525,4.7625,2023-06-06,30,2023-07-21,9.525,205.0


In [425]:
df_opts.dte.sort_values().unique()

array([  2,   7,  11,  16,  20,  25,  30,  50,  69,  94, 114, 133, 155,
       194, 261, 324, 405, 510, 637], dtype=int16)

In [289]:
# For symbol in backtest_candidates,  
# # get daily OHLC options price for given symbol until exit_50_perc is <= mid-price OR date_21dte is reached.
# # if exit_50_perc is <= mid-price is met,  add a row to the exits_df with the date, symbol, cost_basis, premium, exit_50_perc, and exit_price (exit_50_perc)
# # If exit_50_perc is <= mid-price, is not met, continue to get mid-price for each symbol until date_21dte. On date_21dte, get symbol mid for  add a row to the exits_df with the date, symbol, cost_basis, premium, exit_50_perc, and exit_price (mid-price)




In [290]:
def backtest_exit_strategy(backtest_candidates, nyse_calendar, client, config):
    """
    Backtest exit strategy for wheel options
    
    Exit conditions:
    1. Profit target: Exit when mid-price <= 50% of premium (early exit)
       - If daily range contains exit_50_perc, assume we exited at that exact price
    2. Time limit: Force exit at 21 DTE using 15:45 ET price
    
    Args:
        backtest_candidates: DataFrame with options to backtest
        nyse_calendar: NYSE calendar for trading days
        client: Databento client
        config: Configuration dict
    
    Returns:
        DataFrame with exit results
    """
    exits = []
    
    for idx, row in backtest_candidates.iterrows():
        symbol = row['symbol']
        
        # Ensure all dates are Timestamps and normalize timezone
        entry_date = pd.Timestamp(row['date']).tz_localize(None)  # Remove timezone
        expiration = pd.Timestamp(row['expiration']).tz_localize(None)
        date_21dte = pd.Timestamp(row['date_21dte']).tz_localize(None)
        
        # Entry details
        premium = row['mid']  # What we sold it for
        exit_50_perc = premium * 0.50  # 50% profit target
        cost_basis = row['strike'] * 100
        
        print(f"\nProcessing {symbol}...")
        print(f"  Entry: {entry_date.date()}, Premium: ${premium:.2f}")
        print(f"  Exit target: ${exit_50_perc:.2f} (50%)")
        print(f"  21 DTE date: {date_21dte.date()}")
        
        # Fetch daily prices from entry to 21 DTE (exclusive)
        try:
            # Daily prices for monitoring
            start_daily = entry_date + pd.Timedelta(days=1)  # Day after entry
            end_daily = date_21dte  # Up to but not including 21 DTE
            
            daily_data = client.timeseries.get_range(
                dataset='OPRA.PILLAR',
                schema='ohlcv-1d',
                symbols=symbol,
                stype_in='raw_symbol',
                start=start_daily,
                end=end_daily,
            )
            
            df_daily = daily_data.to_df(tz=config['timezone'])
            
            # Check daily for profit target
            profit_target_hit = False
            
            for check_date, daily_row in df_daily.iterrows():
                # Use daily low/high as price range
                daily_low_mid = daily_row['low']
                daily_high_mid = daily_row['high']
                
                # Check if our exit target is within the daily range
                if daily_low_mid <= exit_50_perc <= daily_high_mid:
                    # Profit target was achievable this day!
                    # We assume we exited at exactly exit_50_perc
                    exits.append({
                        'symbol': symbol,
                        'entry_date': entry_date,
                        'exit_date': check_date.tz_localize(None),
                        'expiration': expiration,
                        'cost_basis': cost_basis,
                        'premium': premium,
                        'exit_50_perc': exit_50_perc,
                        'exit_price': exit_50_perc,  # Exact target price
                        'exit_reason': 'profit_target',
                        'days_held': (check_date.tz_localize(None) - entry_date).days,
                        'daily_low': daily_row['low'],
                        'daily_high': daily_row['high'],
                    })
                    
                    print(f"  ✓ Profit target hit on {check_date.date()} @ ${exit_50_perc:.2f}")
                    print(f"    (Daily range: ${daily_row['low']:.2f} - ${daily_row['high']:.2f})")
                    profit_target_hit = True
                    break
            
            # If profit target not hit, force exit at 21 DTE with 15:45 ET price
            if not profit_target_hit:
                # Fetch precise 15:45 ET price on 21 DTE date
                exit_time = pd.Timestamp(date_21dte.date()).tz_localize(config['timezone']).replace(
                    hour=15, minute=45
                )
                
                data_21dte = client.timeseries.get_range(
                    dataset='OPRA.PILLAR',
                    schema='ohlcv-1m',
                    symbols=symbol,
                    stype_in='raw_symbol',
                    start=exit_time,
                    end=exit_time + pd.Timedelta(minutes=1),
                )
                
                df_21dte = data_21dte.to_df(tz=config['timezone'])
                
                if len(df_21dte) > 0:
                    # Use close price at 15:45 ET as exit price
                    exit_price = df_21dte.iloc[0]['close']
                    
                    exits.append({
                        'symbol': symbol,
                        'entry_date': entry_date,
                        'exit_date': date_21dte,
                        'expiration': expiration,
                        'cost_basis': cost_basis,
                        'premium': premium,
                        'exit_50_perc': exit_50_perc,
                        'exit_price': exit_price,
                        'exit_reason': 'time_limit_21dte',
                        'days_held': (date_21dte - entry_date).days,
                        'daily_low': None,
                        'daily_high': None,
                    })
                    
                    print(f"  ⏰ Time limit exit on {date_21dte.date()} @ 15:45 ET: ${exit_price:.2f}")
                else:
                    print(f"  ⚠ No data available for 21 DTE exit at 15:45 ET")
                    
        except Exception as e:
            print(f"  ✗ Error: {e}")
            import traceback
            traceback.print_exc()
            continue
    
    # Create results DataFrame
    exits_df = pd.DataFrame(exits)
    
    # Calculate P&L
    if len(exits_df) > 0:
        exits_df['exit_pnl'] = exits_df['premium'] - exits_df['exit_price']
        exits_df['exit_pnl_pct'] = (exits_df['exit_pnl'] / exits_df['premium']) * 100
        exits_df['roc'] = (exits_df['exit_pnl'] / exits_df['cost_basis']) * 100
    
    return exits_df

# Configuration
CONFIG = {
    'timezone': 'America/New_York',
}

# Run backtest
exits_df = backtest_exit_strategy(
    backtest_candidates=backtest_candidates,
    nyse_calendar=nyse,
    client=client,
    config=CONFIG
)

# Display results
print("\n" + "="*60)
print("BACKTEST RESULTS")
print("="*60)
print(f"\nTotal exits: {len(exits_df)}")

if len(exits_df) > 0:
    print(f"\nExit reasons:")
    print(exits_df['exit_reason'].value_counts())
    print(f"\nP&L Summary:")
    print(exits_df[['exit_pnl', 'exit_pnl_pct', 'roc']].describe())
    
    # Show sample
    print("\nSample exits:")
    print(exits_df[['symbol', 'entry_date', 'exit_date', 'premium', 'exit_price', 
                   'exit_pnl', 'roc', 'exit_reason']].head(10))
else:
    print("\n⚠ No exits recorded - check for errors above")



Processing TSLA  230721P00200000...
  Entry: 2023-06-06, Premium: $7.85
  Exit target: $3.92 (50%)
  21 DTE date: 2023-06-22
  ✓ Profit target hit on 2023-06-08 @ $3.92
    (Daily range: $3.20 - $3.95)

Processing TSLA  230721P00205000...
  Entry: 2023-06-06, Premium: $9.53
  Exit target: $4.76 (50%)
  21 DTE date: 2023-06-22
  ✓ Profit target hit on 2023-06-08 @ $4.76
    (Daily range: $3.90 - $4.80)

BACKTEST RESULTS

Total exits: 2

Exit reasons:
exit_reason
profit_target    2
Name: count, dtype: int64

P&L Summary:
       exit_pnl  exit_pnl_pct       roc
count  2.000000           2.0  2.000000
mean   4.343750          50.0  0.021428
std    0.592202           0.0  0.002550
min    3.925000          50.0  0.019625
25%    4.134375          50.0  0.020527
50%    4.343750          50.0  0.021428
75%    4.553125          50.0  0.022330
max    4.762500          50.0  0.023232

Sample exits:
                  symbol entry_date           exit_date  premium  exit_price  \
0  TSLA  230721P002

In [428]:
exits_df.symbol


0    TSLA  230721P00200000
1    TSLA  230721P00205000
Name: symbol, dtype: object

In [429]:
df_opts.symbol.unique()


array(['TSLA  230721P00220000', 'TSLA  230721P00215000',
       'TSLA  230721P00195000', 'TSLA  230721P00160000',
       'TSLA  230721P00155000', 'TSLA  230714P00210000',
       'TSLA  230721P00190000'], dtype=object)

In [ ]:


def snapshot_cmbp(df: pd.DataFrame, T: pd.Timestamp) -> pd.DataFrame:
    d0 = df.loc[df["ts_event"] <= T].sort_values("ts_event")
    snap = d0.groupby("symbol", as_index=False).tail(1).copy()

    if snap["bid_px_00"].max() > 1e6:  # heuristic: nanodollars will be huge integers
        snap["bid"] = snap["bid_px_00"] * 1e-9
        snap["ask"] = snap["ask_px_00"] * 1e-9
    else:
        snap["bid"] = snap["bid_px_00"].astype(float)
        snap["ask"] = snap["ask_px_00"].astype(float)

    snap["bid_sz"] = snap["bid_sz_00"].astype(float)
    snap["ask_sz"] = snap["ask_sz_00"].astype(float)

    # Quote quality
    snap["mid"] = (snap["bid"] + snap["ask"]) / 2.0
    snap["abs_spread"] = snap["ask"] - snap["bid"]
    snap["rel_spread"] = np.where(snap["mid"] > 0, snap["abs_spread"] / snap["mid"], np.inf)

    # Staleness (how old is this quote vs decision time)
    snap["stale_s"] = (T - snap["ts_event"]).dt.total_seconds()

    return snap


In [ ]:
def liquidity_filter(
    snap: pd.DataFrame,
    *,
    max_rel_spread: float = 0.12,
    min_bid: float = 0.05,
    min_sz: int = 5,
    max_stale_s: float = 2.0,
    require_two_sided: bool = True
) -> pd.DataFrame:

    ok = (
        (snap["bid"] >= min_bid) &
        (snap["ask"] > snap["bid"]) &
        (snap["rel_spread"] <= max_rel_spread) &
        (snap["bid_sz"] >= min_sz) &
        (snap["ask_sz"] >= min_sz) &
        (snap["stale_s"] <= max_stale_s)
    )

    if require_two_sided:
        ok = ok & (snap["bid"] > 0) & (snap["ask"] > 0)

    out = snap.loc[ok].copy()
    return out


In [ ]:
def add_fill_prices(q: pd.DataFrame, *, slip_frac: float = 0.25) -> pd.DataFrame:
    q = q.copy()
    q["sell_fill"] = q["mid"] - slip_frac * q["abs_spread"]   # for selling options
    q["buy_fill"]  = q["mid"] + slip_frac * q["abs_spread"]   # for buying options (BTC)
    return q


In [ ]:
T = pd.Timestamp("2023-06-06 15:45:00", tz="America/New_York")

snap = snapshot_cmbp(df, T)
snap_ok = liquidity_filter(snap, max_rel_spread=0.12, min_sz=5, max_stale_s=2.0)
snap_ok = add_fill_prices(snap_ok, slip_frac=0.25)


In [ ]:
snap_ok

,ts_event,rtype,publisher_id,instrument_id,action,side,price,size,flags,ts_in_delta,...,root,dte,bid,ask,bid_sz,ask_sz,mid,abs_spread,rel_spread,stale_s
ts_recv,,,,,,,,,,,,,,,,,,,,,
2023-06-06 15:45:00.000108703-04:00,2023-06-06 15:44:59.999900672-04:00,177,30,3613,A,B,30.85,65,194,0,...,AAPL,388,30.85,31.1,65.0,82.0,30.975,0.25,0.008071,0.000099


In [ ]:
import databento as db
import pandas as pd

client = db.Historical()

T = pd.Timestamp("2023-06-06 15:45:00", tz="US/Eastern")
start = T - pd.Timedelta(seconds=10)
end   = T + pd.Timedelta(seconds=1)  # optional small buffer

trades = client.timeseries.get_range(
    dataset="XNAS.ITCH",      # <-- replace with your equity dataset
    schema="trades",          # tick trades
    symbols=["AAPL"],
    start=start,
    end=end,
).to_df()

# last trade at or before T
trades = trades[trades["ts_event"] <= T].sort_values("ts_event")
S = float(trades.iloc[-1]["price"])


In [ ]:
trades

,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,ts_in_delta,sequence,symbol
ts_recv,,,,,,,,,,,,,
2023-06-06 19:44:50.485742865+00:00,2023-06-06 19:44:50.485577270+00:00,0,2,30,T,A,0,179.330,51,130,165595,380340584,AAPL
2023-06-06 19:44:50.485792211+00:00,2023-06-06 19:44:50.485625056+00:00,0,2,30,T,A,0,179.330,49,0,167155,380340586,AAPL
2023-06-06 19:44:50.485792211+00:00,2023-06-06 19:44:50.485625056+00:00,0,2,30,T,A,0,179.330,2,130,167155,380340587,AAPL
2023-06-06 19:44:50.591673415+00:00,2023-06-06 19:44:50.591507862+00:00,0,2,30,T,B,0,179.330,100,0,165553,380341630,AAPL
2023-06-06 19:44:50.591673415+00:00,2023-06-06 19:44:50.591507862+00:00,0,2,30,T,B,0,179.330,100,130,165553,380341631,AAPL
2023-06-06 19:44:51.401853501+00:00,2023-06-06 19:44:51.401687298+00:00,0,2,30,T,B,0,179.340,1500,0,166203,380350313,AAPL
2023-06-06 19:44:51.401853501+00:00,2023-06-06 19:44:51.401687298+00:00,0,2,30,T,B,0,179.340,100,130,166203,380350314,AAPL
2023-06-06 19:44:51.402093024+00:00,2023-06-06 19:44:51.401926175+00:00,0,2,30,T,B,0,179.340,6,130,166849,380350337,AAPL
2023-06-06 19:44:51.402093024+00:00,2023-06-06 19:44:51.401926175+00:00,0,2,30,T,B,0,179.340,20,0,166849,380350336,AAPL


In [331]:
client = db.Historical()

# ============================================================================
# Fetch Option Definitions for a Single Day
# ============================================================================

def fetch_option_symbols(ticker, date):
    """
    Fetch all available option symbols for a ticker on a specific date
    
    Args:
        ticker: Stock ticker (e.g., 'AAPL')
        date: Date to fetch symbols for (pd.Timestamp or string)
    
    Returns:
        DataFrame with available option symbols
    """
    print(f"Fetching option symbols for {ticker} on {date}...")
    
    # Create timestamp for 15:45 ET
    date_dt = pd.Timestamp(date)
    start_time = pd.Timestamp(date_dt.date()).tz_localize('America/New_York').replace(hour=15, minute=45)
    end_time = start_time + pd.Timedelta(minutes=1)
    
    print(f"  Query time: {start_time}")
    
    try:
        # Fetch full option chain at this moment
        data = client.timeseries.get_range(
            dataset='OPRA.PILLAR',
            schema='ohlcv-1m',
            symbols=f'{ticker}.OPT',
            stype_in='parent',
            start=start_time,
            end=end_time,
        )
        
        df = data.to_df(tz='America/New_York')
        
        print(f"  ✓ Fetched {len(df)} option contracts")
        
        # Get unique symbols
        unique_symbols = df['symbol'].unique()
        print(f"  ✓ Found {len(unique_symbols)} unique option symbols")
        
        return df
        
    except Exception as e:
        print(f"  ✗ Error: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()

# ============================================================================
# TEST IT
# ============================================================================

ticker = 'TSLA'
date = '2023-06-06'

df_options = fetch_option_symbols(ticker, date)

if len(df_options) > 0:
    print(f"\nSuccess! Got {len(df_options)} option contracts")
    print(f"\nColumns: {df_options.columns.tolist()}")
    print(f"\nSample data:")
    print(df_options[['symbol', 'open', 'high', 'low', 'close', 'volume']].head(10))
    
    # Show unique symbols
    print(f"\nFirst 10 unique symbols:")
    print(df_options['symbol'].unique()[:10])
else:
    print("\n⚠️ No data returned")

Fetching option symbols for TSLA on 2023-06-06...
  Query time: 2023-06-06 15:45:00-04:00
  ✓ Fetched 315 option contracts
  ✓ Found 136 unique option symbols

Success! Got 315 option contracts

Columns: ['rtype', 'publisher_id', 'instrument_id', 'open', 'high', 'low', 'close', 'volume', 'symbol']

Sample data:
                                          symbol    open    high     low  \
ts_event                                                                   
2023-06-06 15:45:00-04:00  TSLA  250117C00150000  100.05  100.05  100.05   
2023-06-06 15:45:00-04:00  TSLA  250117C00220000   63.85   63.85   63.85   
2023-06-06 15:45:00-04:00  TSLA  230623P00212500    5.62    5.62    5.62   
2023-06-06 15:45:00-04:00  TSLA  230915C00250000   14.05   14.05   14.05   
2023-06-06 15:45:00-04:00  TSLA  230721C00225000   15.15   15.15   15.15   
2023-06-06 15:45:00-04:00  TSLA  230616P00190000    0.51    0.51    0.51   
2023-06-06 15:45:00-04:00  TSLA  230609C00205000   15.29   15.29   15.29   
202

In [305]:
df_options.head()

,rtype,publisher_id,instrument_id,open,high,low,close,volume,symbol
ts_event,,,,,,,,,
2023-06-06 15:45:00-04:00,33,26,4623,0.11,0.11,0.11,0.11,4,AAPL 230616C00192500
2023-06-06 15:45:00-04:00,33,29,1753,2.60,2.60,2.60,2.60,1,AAPL 230721C00185000
2023-06-06 15:45:00-04:00,33,21,4198,0.19,0.19,0.19,0.19,31,AAPL 230616C00190000
2023-06-06 15:45:00-04:00,33,28,4198,0.19,0.19,0.19,0.19,14,AAPL 230616C00190000
2023-06-06 15:45:00-04:00,33,31,4198,0.18,0.18,0.18,0.18,2,AAPL 230616C00190000


In [426]:
client = db.Historical()

# ============================================================================
# Fetch Option Definitions for a Single Day
# ============================================================================

def fetch_option_symbols(ticker, date):
    """
    Fetch all available option symbols for a ticker on a specific date
    
    Args:
        ticker: Stock ticker (e.g., 'AAPL')
        date: Date to fetch symbols for (pd.Timestamp or string)
    
    Returns:
        DataFrame with available option symbols
    """
    print(f"Fetching option symbols for {ticker} on {date}...")
    
    # Create timestamp for 15:45 ET
    date_dt = pd.Timestamp(date)
    start_time = pd.Timestamp(date_dt.date()).tz_localize('America/New_York').replace(hour=15, minute=45)
    end_time = start_time + pd.Timedelta(minutes=1)
    
    print(f"  Query time: {start_time}")
    
    try:
        # Fetch full option chain at this moment
        data = client.timeseries.get_range(
            dataset='OPRA.PILLAR',
            schema='ohlcv-1m',
            symbols=f'{ticker}.OPT',
            stype_in='parent',
            start=start_time,
            end=end_time,
        )
        
        df = data.to_df(tz='America/New_York')
        
        print(f"  ✓ Fetched {len(df)} option contracts")
        
        # Get unique symbols
        unique_symbols = df['symbol'].unique()
        print(f"  ✓ Found {len(unique_symbols)} unique option symbols")
        
        return df
        
    except Exception as e:
        print(f"  ✗ Error: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()

# ============================================================================
# TEST IT
# ============================================================================

ticker = 'TSLA'
date = '2023-06-06'

df_options = fetch_option_symbols(ticker, date)

if len(df_options) > 0:
    print(f"\nSuccess! Got {len(df_options)} option contracts")
    print(f"\nColumns: {df_options.columns.tolist()}")
    print(f"\nSample data:")
    print(df_options[['symbol', 'open', 'high', 'low', 'close', 'volume']].head(10))
    
    # Show unique symbols
    print(f"\nFirst 10 unique symbols:")
    print(df_options['symbol'].unique()[:10])
else:
    print("\n⚠️ No data returned")
def parse_expiration_from_symbol(symbol):
    """Extract just the expiration date from OSI symbol"""
    try:
        parts = symbol.split()
        if len(parts) != 2:
            return None
        
        rest = parts[1]
        date_str = rest[:6]  # YYMMDD
        
        year = 2000 + int(date_str[:2])
        month = int(date_str[2:4])
        day = int(date_str[4:6])
        
        return pd.Timestamp(year, month, day)
    except:
        return None

def parse_call_put_from_symbol(symbol):
    """Extract call/put indicator from OSI symbol"""
    try:
        parts = symbol.split()
        if len(parts) != 2:
            return None
        
        rest = parts[1]
        return rest[6]  # 'C' or 'P'
    except:
        return None

# Parse expiration dates and call/put from symbols
df_options['expiration'] = df_options['symbol'].apply(parse_expiration_from_symbol)
df_options['call_put'] = df_options['symbol'].apply(parse_call_put_from_symbol)

# Get the entry date from ts_event (strip timezone)
entry_date = df_options.index[0].tz_localize(None).normalize()

# Calculate date range (30-45 days from entry)
min_expiry = entry_date + pd.Timedelta(days=30)
max_expiry = entry_date + pd.Timedelta(days=45)

print(f"Entry date: {entry_date.date()}")
print(f"Expiry range: {min_expiry.date()} to {max_expiry.date()}")

# Filter options: 30-45 days expiry AND puts only
df_filtered = df_options[
    (df_options['expiration'] >= min_expiry) &
    (df_options['expiration'] <= max_expiry) &
    (df_options['call_put'] == 'P')
]

print(f"\nFiltered from {len(df_options)} to {len(df_filtered)} options")
print(f"  (Puts only, 30-45 days to expiry)")

print(f"\nUnique expiration dates in range:")
print(df_filtered['expiration'].value_counts().sort_index())

# Show sample
print(f"\nSample:")
print(df_filtered[['symbol', 'expiration', 'call_put', 'close', 'volume']].head(10))


client = db.Historical()

dataset = "OPRA.PILLAR"
schema = "cmbp-1"
tz = "America/New_York"

# Get the list of filtered symbols (30-45 day puts)
symbols_to_fetch = df_filtered['symbol'].tolist()

print(f"Fetching market data for {len(symbols_to_fetch)} filtered symbols...")

start = pd.Timestamp("2023-06-06 15:45", tz=tz)
end   = start + pd.Timedelta(minutes=1)

data = client.timeseries.get_range(
    dataset=dataset,
    schema=schema,
    symbols=symbols_to_fetch,    # ✅ Use specific symbol list
    stype_in="raw_symbol",       # ✅ Use raw_symbol for exact symbols
    start=start,
    end=end,
)

df_opts = data.to_df(tz=tz).sort_values("ts_event")

print(f"✓ Fetched {len(df_opts)} option contracts")
print(f"\nSample:")
print(df_opts[['symbol', 'bid_px_00', 'ask_px_00', 'bid_sz_00', 'ask_sz_00']].head(10))



Fetching option symbols for TSLA on 2023-06-06...
  Query time: 2023-06-06 15:45:00-04:00
  ✓ Fetched 315 option contracts
  ✓ Found 136 unique option symbols

Success! Got 315 option contracts

Columns: ['rtype', 'publisher_id', 'instrument_id', 'open', 'high', 'low', 'close', 'volume', 'symbol']

Sample data:
                                          symbol   open   high    low  close  \
ts_event                                                                       
2023-06-06 15:45:00-04:00  TSLA  230714P00210000   9.30   9.30   9.30   9.30   
2023-06-06 15:45:00-04:00  TSLA  230609C00252500   0.09   0.09   0.09   0.09   
2023-06-06 15:45:00-04:00  TSLA  230616C00185000  35.54  35.54  35.54  35.54   
2023-06-06 15:45:00-04:00  TSLA  230609C00242500   0.26   0.26   0.26   0.26   
2023-06-06 15:45:00-04:00  TSLA  230609C00240000   0.34   0.34   0.34   0.34   
2023-06-06 15:45:00-04:00  TSLA  230609C00240000   0.36   0.36   0.36   0.36   
2023-06-06 15:45:00-04:00  TSLA  230609C0024000

In [311]:
df_filtered.symbol.tolist()

['AAPL  230721P00170000',
 'AAPL  230721P00180000',
 'AAPL  230721P00160000',
 'AAPL  230707P00180000',
 'AAPL  230721P00180000',
 'AAPL  230714P00165000']

In [430]:

client = db.Historical()

dataset = "OPRA.PILLAR"
schema = "cmbp-1"
tz = "America/New_York"

# Get the list of filtered symbols (30-45 day puts)
symbols_to_fetch = df_filtered['symbol'].tolist()

print(f"Fetching market data for {len(symbols_to_fetch)} filtered symbols...")

start = pd.Timestamp("2023-06-06 15:45", tz=tz)
end   = start + pd.Timedelta(minutes=1)

data = client.timeseries.get_range(
    dataset=dataset,
    schema=schema,
    symbols=symbols_to_fetch,    # ✅ Use specific symbol list
    stype_in="raw_symbol",       # ✅ Use raw_symbol for exact symbols
    start=start,
    end=end,
)

df_opts = data.to_df(tz=tz).sort_values("ts_event")

print(f"✓ Fetched {len(df_opts)} option contracts")
print(f"\nSample:")
print(df_opts[['symbol', 'bid_px_00', 'ask_px_00', 'bid_sz_00', 'ask_sz_00']].head(10))


Fetching market data for 15 filtered symbols...
✓ Fetched 2545 option contracts

Sample:
                                                    symbol  bid_px_00  \
ts_recv                                                                 
2023-06-06 15:45:00.000115529-04:00  TSLA  230721P00220000      16.05   
2023-06-06 15:45:00.004306566-04:00  TSLA  230721P00220000      16.05   
2023-06-06 15:45:00.004434923-04:00  TSLA  230721P00215000      13.60   
2023-06-06 15:45:00.005006263-04:00  TSLA  230721P00195000       6.35   
2023-06-06 15:45:00.006805709-04:00  TSLA  230721P00195000       6.35   
2023-06-06 15:45:00.007662627-04:00  TSLA  230721P00220000      16.05   
2023-06-06 15:45:00.007969259-04:00  TSLA  230721P00220000      16.05   
2023-06-06 15:45:00.023645876-04:00  TSLA  230721P00220000      16.05   
2023-06-06 15:45:00.026923524-04:00  TSLA  230721P00220000      16.05   
2023-06-06 15:45:00.033385207-04:00  TSLA  230721P00220000      16.05   

                                  

In [368]:
df_opts.shape



(2545, 22)